In [37]:
!pip install torch
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import pandas as pd
from tokenization_kobert import KoBertTokenizer

In [39]:
device = torch.device("cuda")

model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/fine_tuned_model_final", num_labels=4).to(device)
tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")


model.eval()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [41]:
data = pd.read_csv('all_data_final.csv')
data

,comment,label
0,대구 시장 권영진 하는 꼬라지 보니깐,0
1,4일내로 대구에서 추가로 50명돌파할듯. 가족은 빼박 옮겼을테고 주변인들 옮겼겠지,0
2,전라도세끼들은 딱봐도 뭔가 혐오감있게 생김 유전자가 다른 종족인건 확실함,0
3,이 나라에 범죄자가 살기 좋은 나라구나,0
4,아니 근데 막상 시키면 왜이런거 시키냐고 그러고 농땡이부리면서 안하던데,0
...,...,...
13912,똥싸자마자 물 내려야지 . 새어나오는 가스는 솔직히 어쩔 수 없는거고,3
13913,성소수자 이빈 이띤 좆같고 뱡신 같은 말은 누가 만드는거냐?,1
13914,께롱께롱께롱!!!,3
13915,갱상도가아니라 홍어지 개좃같은 홍어년들,2


In [42]:
data['pred_0'] = None
data['pred_1'] = None
data['pred_2'] = None
data['pred_3'] = None

data

,comment,label,pred_0,pred_1,pred_2,pred_3
0,대구 시장 권영진 하는 꼬라지 보니깐,0,None,None,None,None
1,4일내로 대구에서 추가로 50명돌파할듯. 가족은 빼박 옮겼을테고 주변인들 옮겼겠지,0,None,None,None,None
2,전라도세끼들은 딱봐도 뭔가 혐오감있게 생김 유전자가 다른 종족인건 확실함,0,None,None,None,None
3,이 나라에 범죄자가 살기 좋은 나라구나,0,None,None,None,None
4,아니 근데 막상 시키면 왜이런거 시키냐고 그러고 농땡이부리면서 안하던데,0,None,None,None,None
...,...,...,...,...,...,...
13912,똥싸자마자 물 내려야지 . 새어나오는 가스는 솔직히 어쩔 수 없는거고,3,None,None,None,None
13913,성소수자 이빈 이띤 좆같고 뱡신 같은 말은 누가 만드는거냐?,1,None,None,None,None
13914,께롱께롱께롱!!!,3,None,None,None,None
13915,갱상도가아니라 홍어지 개좃같은 홍어년들,2,None,None,None,None


In [43]:
for idx, row in data.iterrows():
    sentence = row['comment']
    encoded_input = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        padding='max_length',
        max_length=128,
        truncation=True,
        return_tensors='pt'
    )
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)


    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        #predicted_labels = torch.argmax(logits, dim=1)
    data.iat[idx,2] = logits[0][0].item()
    data.iat[idx,3] = logits[0][1].item()
    data.iat[idx,4] = logits[0][2].item()
    data.iat[idx,5] = logits[0][3].item()
    if(idx%1000 == 0):
        print(idx)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000


In [52]:
data

,comment,label,pred_0,pred_1,pred_2,pred_3
0,대구 시장 권영진 하는 꼬라지 보니깐,0,3.856386,-0.943687,-2.462979,-1.623028
1,4일내로 대구에서 추가로 50명돌파할듯. 가족은 빼박 옮겼을테고 주변인들 옮겼겠지,0,4.081994,-2.663378,-1.390284,-0.975608
2,전라도세끼들은 딱봐도 뭔가 혐오감있게 생김 유전자가 다른 종족인건 확실함,0,3.893225,-2.770055,-0.878204,-1.169561
3,이 나라에 범죄자가 살기 좋은 나라구나,0,4.103894,-1.447478,-2.922426,-1.016889
4,아니 근데 막상 시키면 왜이런거 시키냐고 그러고 농땡이부리면서 안하던데,0,2.252744,1.179522,-4.325613,-0.709045
...,...,...,...,...,...,...
13912,똥싸자마자 물 내려야지 . 새어나오는 가스는 솔직히 어쩔 수 없는거고,3,3.538437,-1.553697,-4.020338,0.531356
13913,성소수자 이빈 이띤 좆같고 뱡신 같은 말은 누가 만드는거냐?,1,-0.201312,3.902608,-2.948709,-1.930906
13914,께롱께롱께롱!!!,3,-0.078752,-0.594911,-3.936411,3.411217
13915,갱상도가아니라 홍어지 개좃같은 홍어년들,2,-0.924945,-0.969631,4.791058,-2.23512


In [44]:
data.to_csv("all_data_pred.csv",encoding = 'cp949', index = False)

In [74]:
import BackGround as bg

In [75]:
ori_label = data['label']
pred_label = list(zip(data['pred_0'],data['pred_1'],data['pred_2'],data['pred_3']))
pred_label = [[a,b,c,d] for a,b,c,d in pred_label]
print(ori_label[:10])
print(pred_label[:10])

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: label, dtype: int64
[[3.8563859462738037, -0.9436872005462646, -2.462979316711426, -1.6230281591415405], [4.08199405670166, -2.6633782386779785, -1.3902835845947266, -0.9756084084510803], [3.8932254314422607, -2.770055055618286, -0.8782044053077698, -1.1695610284805298], [4.103893756866455, -1.4474776983261108, -2.9224259853363037, -1.0168886184692383], [2.252744197845459, 1.1795215606689453, -4.325613498687744, -0.7090445160865784], [3.977104902267456, -1.8175430297851562, -1.3865435123443604, -1.8746072053909302], [3.9791958332061768, -2.683412790298462, -1.0339055061340332, -1.1209805011749268], [4.126907825469971, -1.050250768661499, -2.9128589630126953, -1.5133591890335083], [4.080319881439209, -2.1112215518951416, -1.4280632734298706, -1.5604811906814575], [3.621373176574707, -1.0289345979690552, -3.965055465698242, -0.10459206253290176]]


In [76]:
openmax = bg.OpenMax()
openmax.fit(pred_label,ori_label)

In [77]:
def predict_sentence(sentence):
    encoded_input = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        padding='max_length',
        max_length=128,
        truncation=True,
        return_tensors='pt'
    )
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)


    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = [0,0,0,0]
    logits[0] = outputs.logits[0][0].item()
    logits[1] = outputs.logits[0][1].item()
    logits[2] = outputs.logits[0][2].item()
    logits[3] = outputs.logits[0][3].item()

    om_result = openmax.calculate(logits)
    _class = ['혐오','성차별','일베']
    print("입력문장 : ",sentence)
    print()
    print("모델 예측 결과 - OpenMax 미적용")
    print("혐오 : ",logits[0])
    print("성차별 : ",logits[1])
    print("일베 : ",logits[2])
    print("Unknown : ",logits[3])

    print()
    print("모델 예측 결과 - OpenMax 적용")
    print("혐오 : ",om_result[0])
    print("성차별 : ",om_result[1])
    print("일베 : ",om_result[2])
    print("Unknown : ",om_result[3])


In [78]:
predict_sentence("철수하고 영희가 집에 간다")

입력문장 :  철수하고 영희가 집에 간다

모델 예측 결과 - OpenMax 미적용
혐오 :  0.705388069152832
성차별 :  -1.7096636295318604
일베 :  -3.597870111465454
Unknown :  3.3022656440734863

모델 예측 결과 - OpenMax 적용
혐오 :  0.0345714071041379
성차별 :  0.0333566408328471
일베 :  0.033356314638108224
Unknown :  0.8987156374249067


In [ ]:
openmax.calculate([1,1,1])

[0.11069866539568819,
 0.29422751187099094,
 0.2975303280194986,
 0.2975434947138223]

In [51]:
data['label'].value_counts()

1    4481
0    4172
3    3739
2    1525
Name: label, dtype: int64